Derive Orbital Elements
<br>
<br>
    - Semi-major axis: The distance from the center of an ellipse to its furthest edge, along the major axis
<br>    
    - Eccentricity: Measure of how elliptical the orbit is around the sun, and is represented by a value between 0 and 1: 
<br>
    Eccentricity of 0: A perfect circle 
<br>
    Eccentricity of 1: A parabola
<br>
<br>
    - Inclination: The angle between the plane of the orbit and the ecliptic plane(Plane of Earth's orbit around the sun)
<br>
    - Longitude of the ascending node: The angle between the reference direction and the point at which the orbit crosses the ecliptic plane from the south to the north

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('RawData/horizons_results_30_years.csv')

GM = np.float128(1.32712440018e11) # km^3/s^2

df.head()

,Date,Position_x,Position_y,Position_z,Velocity_x,Velocity_y,Velocity_z
0,1964-09-01,9.802199e+07,4.642303e+07,-5.009535e+06,-16.841095,31.169176,1.515031
1,1964-09-02,9.653499e+07,4.907756e+07,-4.878073e+06,-17.559232,30.280814,1.525763
2,1964-09-03,9.499122e+07,5.165641e+07,-4.746249e+06,-18.157275,29.421917,1.523812
3,1964-09-04,9.340043e+07,5.416325e+07,-4.615044e+06,-18.650868,28.617836,1.511939
4,1964-09-05,9.177075e+07,5.660374e+07,-4.485177e+06,-19.061267,27.889125,1.493449


In [2]:
position = df[['Position_x', 'Position_y', 'Position_z']].values.astype(np.float128)
velocity = df[['Velocity_x', 'Velocity_y', 'Velocity_z']].values.astype(np.float128)

angular_momentum = np.cross(position, velocity)
vel_cross_h = np.cross(velocity, angular_momentum)

angular_momentum_magnitude = np.sqrt(angular_momentum[:,0]**2 + angular_momentum[:,1]**2 + angular_momentum[:,2]**2)
pos_magnitude = np.sqrt(np.sum(position**2, axis=1, dtype=np.float128))

In [3]:
#Caclulate semi-major axis
pos_x = df['Position_x'].values
pos_y = df['Position_y'].values
pos_z = df['Position_z'].values
vel_x = df['Velocity_x'].values
vel_y = df['Velocity_y'].values
vel_z = df['Velocity_z'].values

r_magnitude = np.sqrt(pos_x**2 + pos_y**2 + pos_z**2)
v_magnitude = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)

semi_major_axis_arr = 1/((2/r_magnitude) - (v_magnitude**2/GM))

semi_major_axis = np.mean(semi_major_axis_arr)

print('Semi-major axis: ', semi_major_axis)
print("True value of semi-major axis of Venus: 108200000 km")
print("Relative error: ", ((semi_major_axis - 108200000)/108200000)*100, "%")

Semi-major axis:  109444031.56288806335
True value of semi-major axis of Venus: 108200000 km
Relative error:  1.1497519065508903439 %


In [4]:
# #Calculate eccentricity

# epsilon = (v_magnitude**2)/2 - GM/r_magnitude

# eccentricity_arr = np.sqrt(1 + (2*epsilon*angular_momentum_magnitude**2)/(GM**2))
# eccentricity_arr

In [ ]:
print(np.sum(position[0]**2))
x = np.sqrt(np.sum(position[0]**2))
print(x)
y = position[0][0]/x
print(y)
np.sum(position[0]/np.sqrt(np.sum(position[0]**2))**2)

# (position/pos_magnitude[:,None])[0]

1.1788504132939676087e+16
108574878.001035193825
0.9028054578143003174


array([ 0.90280546,  0.42756695, -0.04613898], dtype=float128)

In [ ]:
def calculate_eccentricity_stable(row):
    r = np.array([row["Position_x"], row["Position_y"], row["Position_z"]])
    v = np.array([row["Velocity_x"], row["Velocity_y"], row["Velocity_z"]])
    r_mag = np.sqrt(np.sum(r**2))

    h_vec = np.cross(r, v)

    mu = 1.32712440018 * 10**20

    eccentricity_vector = (np.cross(v, h_vec) / mu) - (r / r_mag)
    # eccentricity = np.sqrt(np.sum(eccentricity_vector**2))
    return eccentricity_vector

df["Eccentricity"] = df.apply(calculate_eccentricity_stable, axis=1)
df['Eccentricity']


array([-0.90280546, -0.42756695,  0.04613898])

In [64]:
a = np.array([ 9.01918320e-10,  4.89507524e-10, -4.50512292e-11])
b = np.array([ 0.90280546,  0.42756695, -0.04613898])
a-b

array([-0.90280546, -0.42756695,  0.04613898])

In [5]:
# #Calculate eccentricity
# position = df[['Position_x', 'Position_y', 'Position_z']].values.astype(np.float128)
# velocity = df[['Velocity_x', 'Velocity_y', 'Velocity_z']].values.astype(np.float128)

# eccentricity_vec = (vel_cross_h/GM) - (position/pos_magnitude[:, None])
# #magnitude of eccentricity_vec
# eccentricity = np.sqrt(np.sum(eccentricity_vec**2, axis=1, dtype=np.float128))

# print('Eccentricity at each time step: ', eccentricity)
# print('Eccentricity: ', np.mean(eccentricity))
# print("True value of eccentricity of Venus: 0.007")
# print("Relative error: ", ((np.mean(eccentricity) - 0.007)/0.007)*100, "%")

In [6]:
#calcculate average inclination
inclination = np.arccos(angular_momentum[:,2]/angular_momentum_magnitude)
inclination = np.degrees(inclination)
inclination = np.mean(inclination)
print('Inclination: ', inclination)
print("True value of inclination of Venus: 3.39 degrees")
print("Relative error: ", ((inclination - 3.39)/3.39)*100, "%")

Inclination:  3.3941502647263799737
True value of inclination of Venus: 3.39 degrees
Relative error:  0.12242668809380086163 %


In [8]:
#Longitude of ascending node
omega = np.arctan2(angular_momentum[:,0], -1*angular_momentum[:,1])
omega = np.degrees(omega)
omega = np.mean(omega)
print('Longitude of ascending node: ', omega)
print("True value of Longitude of ascending node of Venus: 76.68 degrees")
print("Relative error: ", ((omega - 76.68)/76.68)*100, "%")

Longitude of ascending node:  76.7382274726436536
True value of Longitude of ascending node of Venus: 76.68 degrees
Relative error:  0.07593567115759881576 %
